In [2]:
import os
from params import PARAMS
import pickle
import numpy as np
import json
import pandas as pd
from main import show_metrics

In [4]:
final_results = pd.read_csv('../output_400_105_subjects_with_gcram/results.csv')
final_results.groupby(['model_name']).mean()

,random_seed,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params
model_name,,,,,,,,
imagine_cnn,599479.0,0.321984,0.324646,0.324883,0.321709,0.321984,0.547830,2828820.0
imagine_fcn,599479.0,0.279687,0.221950,0.221829,0.281028,0.279687,0.520697,431108.0
imagine_gat_auto,599479.0,0.260484,0.201732,0.201365,0.262279,0.260484,0.508192,436228.0
imagine_gcn,599479.0,0.248828,0.078752,0.078164,0.251061,0.248828,0.500709,431108.0
imagine_gcn_auto,599479.0,0.300938,0.300976,0.301199,0.300916,0.300938,0.533960,435204.0
imagine_gcram,599479.0,0.326078,0.334564,0.334701,0.325521,0.326078,0.550369,558616.0
imagine_gcram_auto,599479.0,0.276703,0.212702,0.212472,0.277113,0.276703,0.518059,562712.0
imagine_rnn,599479.0,0.258672,0.258677,0.258811,0.258631,0.258672,0.505755,1201156.0


In [5]:
std_per_dataset = final_results.groupby(['model_name', 'dataset_name']).std().drop(columns=['random_seed'])
std_per_dataset = std_per_dataset.rename(columns={'accuracy': 'accuracy_std',
                    'precision_macro': 'precision_macro_std', 
                    'precision_weighted': 'precision_weighted_std', 
                    'recall_macro': 'recall_macro_std', 
                    'recall_weighted': 'recall_weighted_std'})
std_per_dataset = std_per_dataset.drop(columns=['AUROC', 'n_params'])
mean_per_dataset = final_results.groupby(['model_name', 'dataset_name']).mean().drop(columns='random_seed')
results_per_dataset = pd.concat([mean_per_dataset, std_per_dataset], axis=1)
results_per_dataset.reset_index()

,model_name,dataset_name,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params,accuracy_std,precision_macro_std,precision_weighted_std,recall_macro_std,recall_weighted_std
0,imagine_cnn,cross_subject_data_0,0.320156,0.321318,0.321716,0.319250,0.320156,0.546230,2828820.0,0.010256,0.008402,0.008695,0.009649,0.010256
1,imagine_cnn,cross_subject_data_1,0.329141,0.330786,0.331075,0.328453,0.329141,0.552333,2828820.0,0.014953,0.014168,0.014268,0.016753,0.014953
2,imagine_cnn,cross_subject_data_2,0.316641,0.319742,0.319749,0.316268,0.316641,0.544175,2828820.0,0.004240,0.006289,0.006034,0.004135,0.004240
3,imagine_cnn,cross_subject_data_3,0.320547,0.325967,0.326148,0.321372,0.320547,0.547575,2828820.0,0.008112,0.012275,0.012585,0.008846,0.008112
4,imagine_cnn,cross_subject_data_4,0.323437,0.325415,0.325725,0.323203,0.323437,0.548837,2828820.0,0.012244,0.011410,0.011563,0.012380,0.012244
5,imagine_fcn,cross_subject_data_0,0.262500,0.152357,0.152159,0.264270,0.262500,0.509526,431108.0,0.021117,0.124015,0.124665,0.019754,0.021117
6,imagine_fcn,cross_subject_data_1,0.284609,0.248869,0.249084,0.285174,0.284609,0.523486,431108.0,0.022155,0.104815,0.105316,0.020964,0.022155
7,imagine_fcn,cross_subject_data_2,0.277031,0.204878,0.204263,0.279647,0.277031,0.519757,431108.0,0.032052,0.131799,0.132665,0.028781,0.032052
8,imagine_fcn,cross_subject_data_3,0.281016,0.207916,0.207751,0.282582,0.281016,0.521742,431108.0,0.031735,0.133598,0.134282,0.029773,0.031735
9,imagine_fcn,cross_subject_data_4,0.293281,0.295731,0.295888,0.293468,0.293281,0.528974,431108.0,0.006210,0.006059,0.006108,0.006076,0.006210


In [6]:
std_per_model = final_results.groupby(['model_name']).std().drop(columns=['random_seed'])
std_per_model = std_per_model.rename(columns={'accuracy': 'accuracy_std',
                    'precision_macro': 'precision_macro_std', 
                    'precision_weighted': 'precision_weighted_std', 
                    'recall_macro': 'recall_macro_std', 
                    'recall_weighted': 'recall_weighted_std'})
std_per_model = std_per_model.drop(columns=['AUROC', 'n_params'])
mean_per_model = final_results.groupby(['model_name']).mean().drop(columns='random_seed')
results_per_model = pd.concat([mean_per_model, std_per_model], axis=1)
results_per_model

,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params,accuracy_std,precision_macro_std,precision_weighted_std,recall_macro_std,recall_weighted_std
model_name,,,,,,,,,,,,
imagine_cnn,0.321984,0.324646,0.324883,0.321709,0.321984,0.547830,2828820.0,0.010580,0.010690,0.010832,0.011005,0.010580
imagine_fcn,0.279687,0.221950,0.221829,0.281028,0.279687,0.520697,431108.0,0.024669,0.112544,0.113127,0.022923,0.024669
imagine_gat_auto,0.260484,0.201732,0.201365,0.262279,0.260484,0.508192,436228.0,0.014307,0.098894,0.099673,0.011683,0.014307
imagine_gcn,0.248828,0.078752,0.078164,0.251061,0.248828,0.500709,431108.0,0.008435,0.058743,0.059099,0.005171,0.008435
imagine_gcn_auto,0.300938,0.300976,0.301199,0.300916,0.300938,0.533960,435204.0,0.011316,0.011225,0.011293,0.011266,0.011316
imagine_gcram,0.326078,0.334564,0.334701,0.325521,0.326078,0.550369,558616.0,0.022794,0.032972,0.033222,0.023565,0.022794
imagine_gcram_auto,0.276703,0.212702,0.212472,0.277113,0.276703,0.518059,562712.0,0.017400,0.083207,0.083292,0.017722,0.017400
imagine_rnn,0.258672,0.258677,0.258811,0.258631,0.258672,0.505755,1201156.0,0.007581,0.007618,0.007555,0.007708,0.007581


In [ ]:
path = 'output/imagine_cnn/cross_subject_data_0/195645/results.pickle'
results = pickle.load(open(path, 'rb'))

In [ ]:
results['history']['val_acc']

[0.28912869095802307,
 0.2906447947025299,
 0.30393293499946594,
 0.29135823249816895,
 0.28877195715904236,
 0.29626327753067017,
 0.2916257977485657,
 0.294568806886673,
 0.2949255406856537,
 0.30009809136390686,
 0.2949255406856537,
 0.29474717378616333,
 0.2933202385902405,
 0.30214929580688477,
 0.29082316160202026,
 0.28029966354370117,
 0.29296353459358215,
 0.2868099510669708,
 0.29465797543525696,
 0.29421207308769226,
 0.2916257977485657,
 0.2868991196155548,
 0.2931418716907501,
 0.29055559635162354,
 0.2914474308490753,
 0.28377774357795715,
 0.2889503240585327,
 0.29055559635162354,
 0.28841522336006165,
 0.29251760244369507,
 0.29180416464805603,
 0.2860964834690094,
 0.28663158416748047,
 0.2899313271045685,
 0.2874342203140259,
 0.2857397496700287,
 0.29171496629714966,
 0.28288593888282776,
 0.29091233015060425,
 0.28636404871940613,
 0.2914474308490753,
 0.2894854247570038,
 0.2901988625526428,
 0.2857397496700287,
 0.2838669419288635,
 0.28618568181991577,
 0.2915365